In [4]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import torch
import os

# Fixing data
0) Name the images correctly(structured)
1) Manualy labeld a batch of data on Roboflow
2) Downloading the labeld data to train localy
3) Labeling all the images with the trained model
4) Putting the labeld images back in Roboflow to go over them. If necessary add labels
5) Then downloading fully correct labeld data to put train and evaluate our model

# Code for changing picture names in a map

In [4]:
# Map path
map_pad = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06")

# Take all pngs and sort them
png_bestanden = sorted(map_pad.glob("*.png"))

print(f"Founded PNG's: {len(png_bestanden)}")

for i, bestand in enumerate(png_bestanden, start=1):
    nieuwe_naam = f"Town06_{i}.png"
    nieuw_pad = map_pad / nieuwe_naam

    print(f"Rename: {bestand.name}  ->  {nieuwe_naam}")
    bestand.rename(nieuw_pad)

print("Everything renamed successfully.")

Founded PNG's: 200
Rename: Town06_1024_320_48.665_1.548_-5.180.png  ->  Town06_1.png
Rename: Town06_1024_320_48.864_1.886_-10.093.png  ->  Town06_2.png
Rename: Town06_1024_320_48.888_1.872_-13.029.png  ->  Town06_3.png
Rename: Town06_1024_320_48.921_1.568_-1.310.png  ->  Town06_4.png
Rename: Town06_1024_320_49.022_1.057_-8.517.png  ->  Town06_5.png
Rename: Town06_1024_320_49.258_1.235_4.533.png  ->  Town06_6.png
Rename: Town06_1024_320_49.259_1.079_3.299.png  ->  Town06_7.png
Rename: Town06_1024_320_49.303_1.127_-6.574.png  ->  Town06_8.png
Rename: Town06_1024_320_49.314_1.786_-4.497.png  ->  Town06_9.png
Rename: Town06_1024_320_49.648_1.124_2.095.png  ->  Town06_10.png
Rename: Town06_1024_320_49.914_1.543_-8.965.png  ->  Town06_11.png
Rename: Town06_1024_320_49.949_1.261_-14.551.png  ->  Town06_12.png
Rename: Town06_1024_320_50.461_1.166_-12.367.png  ->  Town06_13.png
Rename: Town06_1024_320_50.512_1.201_-8.019.png  ->  Town06_14.png
Rename: Town06_1024_320_50.843_1.237_-9.400.png  ->

In [3]:
# Path to the map for the yaml file
BASE_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\DataSet440Labeld")
DATA_YAML = BASE_DIR / "data.yaml"
# Checking if it is there
print(DATA_YAML, DATA_YAML.exists())

C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\DataSet440Labeld\data.yaml True


Training localy

In [4]:
# Checking for GPU
DEVICE = 0 if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

model = YOLO("yolo11s.pt")
# Next: train 13 and val11
results = model.train(
    data=str(DATA_YAML),
    epochs=40,          # Amount epochs
    imgsz=640,
    batch=-1,           # Batch size chosen automaticly
    patience=10,        # early stopping, stop if val loss is not getting better
    workers=4,          # worker amount
    device=DEVICE,      # 0 = GPU else cpu
    plots=False,        # there are some errors with plotting so we disable is for now
)

Using device: 0
Ultralytics 8.3.235  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\DataSet440Labeld\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train13, nbs=64, nms=False, opset=None, o

# Fill in the correct paths and Check if the maps exist

In [13]:
# Paths
BASE_DIR   = Path(r"C:/Users/Khatc/Desktop/DistributedAI/Project/Code_Train")
MODEL_PATH = BASE_DIR / "runs/detect/train13/weights/best.pt"  # Path to the trained model
IMAGES_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06")  # Path for the images that we wanna label
print("Model:", MODEL_PATH, "| exists:", MODEL_PATH.exists())
print("Images dir:", IMAGES_DIR, "| exists:", IMAGES_DIR.exists())

best.pt bestaat: True
data.yaml bestaat: True
Using device: 0


# Here is the code that auto labels the unlabeld images.

In [10]:
from ultralytics import YOLO
# Load trained model
model = YOLO(str(MODEL_PATH))

# Model labeling the pictures
results = model.predict(
    source=str(IMAGES_DIR), # Map where the images are that need to be labeld
    imgsz=640,
    conf=0.3,               # Confidence threshold
    iou=0.5,
    save_txt=True,          # Making YOLO .txt labels
    save=True,              # Saving the labeld images to see how it did
    show=False,
    project=str(BASE_DIR / "autolabel"),  # output map
    name="saved_labels_6",
    exist_ok=True,
)


image 1/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_1.png: 224x640 1 car, 51.6ms
image 2/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_10.png: 224x640 2 cars, 33.4ms
image 3/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_100.png: 224x640 (no detections), 9.3ms
image 4/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_101.png: 224x640 (no detections), 12.2ms
image 5/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_102.png: 224x640 (no detections), 11.0ms
image 6/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_103.png: 224x640 (no detections), 11.0ms
image 7/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_104.png: 224x640 2 cars, 10.9ms
image 8/200 C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town06\Town06_105.png: 224x640 1 car, 8.9ms
image 9/20

We can see that there are many pictures that dont have a .txt file(label). This means that the model has not seen a object on the missing images label. So we still want an empty txt file with the coresponding name off the picture so we can give this to roboflow. The code below makes empty txt file for the missing images.

In [8]:
from pathlib import Path

IMAGES_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code\Data\Images\Town01")   # map with all the images
LABELS_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\autolabel\saved_labels\labels")   # map with all the labels

IMG_EXTS = {".png"}

LABELS_DIR.mkdir(parents=True, exist_ok=True)

created = 0
checked = 0

# Go through all the images
for img_path in IMAGES_DIR.rglob("*"):
    if img_path.suffix.lower() not in IMG_EXTS:
        continue

    checked += 1
    label_path = LABELS_DIR / f"{img_path.stem}.txt"

    # If there is no label, create an empty one
    if not label_path.exists():
        label_path.write_text("", encoding="utf-8")
        created += 1

print(f"Checkte images: {checked}")
print(f"Empty .txt files made: {created}")
print(f"Labels map: {LABELS_DIR}")

Checkte images: 5053
Empty .txt files made: 2572
Labels map: C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\autolabel\saved_labels\labels


# Final code to train our model on a bigger dataset, we are actually "fine tuning" the one we trained with the smaller self-labeled dataset.

In [1]:
# Path to the map for the yaml file
BEST_PT = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect\finetune_3450_from_train13\weights\best.pt")
DATA_YAML = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Final_Dataset_3450\data.yaml")

print("best.pt bestaat:", BEST_PT.exists())
print("data.yaml bestaat:", DATA_YAML.exists())

# Device used
DEVICE = 0 if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

best.pt bestaat: True
data.yaml bestaat: True
Using device: 0


This training below failed because of a version mismatch for the graphing libraries

In [3]:
from ultralytics import YOLO

model = YOLO(str(BEST_PT))

results = model.train(
    data=str(DATA_YAML),
    epochs=60,          # goede balans: accuraat maar niet te lang
    imgsz=640,          # iamge size, auto resize
    batch=-1,           # auto batch size
    patience=20,        # early stopping: enables early stopping by ending training if validation performance doesn’t improve for 20 consecutive epochs. it helps with: saving time and reducing overfitting.
    workers=4,
    device=DEVICE,
    plots=True,         # plotting graphs
    project=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect",
    name="finetune_3450_from_train13"
)



New https://pypi.org/project/ultralytics/8.3.237 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.235  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Final_Dataset_3450\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Users\Khatc\Des

AttributeError: module 'matplotlib' has no attribute '__version_info__'

In [2]:
from ultralytics import YOLO

model = YOLO(str(BEST_PT))

results = model.train(
    data=str(DATA_YAML),
    epochs=60,          # good balance
    imgsz=640,          # iamge size, auto resize
    batch=-1,           # auto batch size
    patience=20,        # early stopping: enables early stopping by ending training if validation performance doesn’t improve for 20 consecutive epochs. it helps with: saving time and reducing overfitting.
    workers=4,
    device=DEVICE,
    plots=True,         # plotting graphs
    project=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect",
    name="finetune_3450_1"
)

New https://pypi.org/project/ultralytics/8.3.237 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.235  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Final_Dataset_3450\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Users\Khatc\Des